In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import collections

In [2]:
%%time
raw_data = pd.read_csv(r"C:\Users\XXXX\Downloads\hotel_review_in_europe\Hotel_Reviews.csv")
raw_data = raw_data.drop_duplicates()[:10000]
print(len(raw_data))

10000
Wall time: 2.74 s


In [3]:
raw_data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from nltk.stem import PorterStemmer
from nltk import ngrams
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))

In [5]:
individual_review_drop_set = set(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date', 'Average_Score',\
                                  'Hotel_Name', 'Reviewer_Nationality', 'Review_Total_Negative_Word_Counts', \
                                  'Total_Number_of_Reviews', 'Review_Total_Positive_Word_Counts', \
                                  'Total_Number_of_Reviews_Reviewer_Has_Given', 'days_since_review', 'lat', 'lng'])
individual_review_raw_data = raw_data.drop(list(individual_review_drop_set), 1)

# Remove stopwords
individual_review_raw_data['Positive_Review'] = individual_review_raw_data['Positive_Review'].apply(
    lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))
individual_review_raw_data['Negative_Review'] = individual_review_raw_data['Negative_Review'].apply(
    lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))

# Stemming
ps = PorterStemmer()
individual_review_raw_data['Positive_Review'] = individual_review_raw_data['Positive_Review'].apply(
    lambda x: ' '.join([ps.stem(y) for y in x.split()]))
individual_review_raw_data['Negative_Review'] = individual_review_raw_data['Negative_Review'].apply(
    lambda x: ' '.join([ps.stem(y) for y in x.split()]))

X = individual_review_raw_data.drop(["Reviewer_Score"], 1)
y = individual_review_raw_data["Reviewer_Score"]
raw_X_train, raw_X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=310)
raw_X_train.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,Negative_Review,Positive_Review,Tags
208,neg nois build work go time awar book stay hug...,staff extrem polit help honestli fault room cl...,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
1682,noth,good locat help staff room spaciou well worth ...,"[' Leisure trip ', ' Group ', ' Superior King ..."
204,neg,good locat next tram go major revamp nois prob...,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
150,sofa need bar better breakfast option water pr...,locat great us close sister alway someon desk ...,"[' Leisure trip ', ' Group ', ' Duplex Double ..."
6761,room took warm put air con high temp heat,staff friendli help,"[' Leisure trip ', ' Couple ', ' Superior Doub..."


In [6]:
raw_X_test.head()

,Negative_Review,Positive_Review,Tags
8374,notic lack face cloth bathroom guess british c...,breakfast buffet excel staff attent staff over...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe..."
9011,room suppos superior doubl room window open pr...,breafast good person also good cater peopl eat...,"[' Couple ', ' Superior Double Room ', ' Staye..."
1144,neg,great staff arriv delay wait readi 3am upgrad ...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe..."
9409,room price bit steep even fine includ breakfas...,hotel beauti amaz antiqu recept hall entir exp...,"[' Leisure trip ', ' Couple ', ' Superior Doub..."
6250,neg,excel locat 5 min walk london eye well maintai...,"[' Leisure trip ', ' Family with young childre..."


In [7]:
%%time
# NGram
ngram_cnt = collections.Counter(
    [a for a in ngrams(" | ".join(raw_X_train['Positive_Review']).split(), 1)] + \
    [a for a in ngrams(" | ".join(raw_X_train['Positive_Review']).split(), 2)] + \
    [a for a in ngrams(" | ".join(raw_X_train['Positive_Review']).split(), 3)] + \
    [a for a in ngrams(" | ".join(raw_X_train['Negative_Review']).split(), 1)] + \
    [a for a in ngrams(" | ".join(raw_X_train['Negative_Review']).split(), 2)] + \
    [a for a in ngrams(" | ".join(raw_X_train['Negative_Review']).split(), 3)])
print([(k, c) for k, c in ngram_cnt.most_common(10) if '|' not in k])
print(len(ngram_cnt))
features = [" ".join(k) for k, _ in ngram_cnt.most_common(1000) if '|' not in k]

[(('room',), 7747), (('staff',), 3858), (('hotel',), 3413), (('locat',), 3349), (('breakfast',), 2054), (('good',), 2007), (('neg',), 1779), (('bed',), 1768)]
236631
Wall time: 251 ms


In [8]:
%%time
# Build features
X_train = raw_X_train.copy()
X_test = raw_X_test.copy()
for f in features:
    X_train["feature_p_" + "__".join(f.split())] = raw_X_train['Positive_Review'].apply(lambda x: f in x)
    X_train["feature_n_" + "__".join(f.split())] = raw_X_train['Negative_Review'].apply(lambda x: f in x)
    
    X_test["feature_p_" + "__".join(f.split())] = raw_X_test['Positive_Review'].apply(lambda x: f in x)
    X_test["feature_n_" + "__".join(f.split())] = raw_X_test['Negative_Review'].apply(lambda x: f in x)
X_train.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

Wall time: 7.27 s


,Negative_Review,Positive_Review,Tags,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,...,feature_p_nearli,feature_n_nearli,feature_p_b,feature_n_b,feature_p_soak,feature_n_soak,feature_p_pm,feature_n_pm,feature_p_disabl,feature_n_disabl
208,neg nois build work go time awar book stay hug...,staff extrem polit help honestli fault room cl...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",True,False,True,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
1682,noth,good locat help staff room spaciou well worth ...,"[' Leisure trip ', ' Group ', ' Superior King ...",True,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
204,neg,good locat next tram go major revamp nois prob...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
150,sofa need bar better breakfast option water pr...,locat great us close sister alway someon desk ...,"[' Leisure trip ', ' Group ', ' Duplex Double ...",True,True,False,False,False,False,True,...,False,False,True,True,False,False,False,False,False,False
6761,room took warm put air con high temp heat,staff friendli help,"[' Leisure trip ', ' Couple ', ' Superior Doub...",False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
X_test.head()

,Negative_Review,Positive_Review,Tags,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,...,feature_p_nearli,feature_n_nearli,feature_p_b,feature_n_b,feature_p_soak,feature_n_soak,feature_p_pm,feature_n_pm,feature_p_disabl,feature_n_disabl
8374,notic lack face cloth bathroom guess british c...,breakfast buffet excel staff attent staff over...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",False,True,True,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False
9011,room suppos superior doubl room window open pr...,breafast good person also good cater peopl eat...,"[' Couple ', ' Superior Double Room ', ' Staye...",False,True,False,False,False,True,False,...,False,False,True,True,False,False,False,False,False,False
1144,neg,great staff arriv delay wait readi 3am upgrad ...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",True,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9409,room price bit steep even fine includ breakfas...,hotel beauti amaz antiqu recept hall entir exp...,"[' Leisure trip ', ' Couple ', ' Superior Doub...",True,True,True,False,True,False,False,...,False,False,True,True,False,False,False,False,False,False
6250,neg,excel locat 5 min walk london eye well maintai...,"[' Leisure trip ', ' Family with young childre...",False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [10]:
X_train = X_train.drop(["Positive_Review", "Negative_Review", "Tags"], 1)
X_train.head()

X_test = X_test.drop(["Positive_Review", "Negative_Review", "Tags"], 1)
X_test.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,feature_n_locat,feature_p_breakfast,feature_n_breakfast,...,feature_p_nearli,feature_n_nearli,feature_p_b,feature_n_b,feature_p_soak,feature_n_soak,feature_p_pm,feature_n_pm,feature_p_disabl,feature_n_disabl
8374,False,True,True,False,False,False,False,False,True,False,...,False,False,True,True,False,False,False,False,False,False
9011,False,True,False,False,False,True,False,False,False,True,...,False,False,True,True,False,False,False,False,False,False
1144,True,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9409,True,True,True,False,True,False,False,False,False,True,...,False,False,True,True,False,False,False,False,False,False
6250,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
%%time
# Fit random forest model
rf_model = RandomForestRegressor(n_estimators=500, max_depth=25, n_jobs=-1, min_samples_split=10, min_samples_leaf=2) # 1.6
# rf_model = RandomForestRegressor(n_estimators=5000, max_depth=15, n_jobs=-1, min_samples_split=5, min_samples_leaf=2) # 1.63
rf_model.fit(X_train, y_train)

Wall time: 4min 12s


RandomForestRegressor(max_depth=25, min_samples_leaf=2, min_samples_split=10,
                      n_estimators=500, n_jobs=-1)

In [12]:
import random
# check against our baseline to make sure the model learns things
r = []
for i in range(len(y_test)):
    n = random.randint(1,10)
    r.append(n)
print("baseline msr:", mean_squared_error(r, y_test.astype('float')))
print("model msr:", mean_squared_error(rf_model.predict(X_test), y_test.astype('float')))

baseline msr: 18.542330000000003
model msr: 1.5652117351801635


In [15]:
# Extract model importance
fi = rf_model.feature_importances_
imp_fea = []
for i in range(len(fi)):
    imp_fea.append((X_train.columns[i], fi[i]))
imp_fea = sorted(imp_fea, key=lambda t: t[1], reverse=True)
print(len(imp_fea))
imp_fea

3316


[('feature_n_room', 0.20181664150728615),
 ('feature_p_e', 0.11350931812595522),
 ('feature_n_c', 0.04374767376990346),
 ('feature_n_st', 0.02590620887322868),
 ('feature_p_staff', 0.01938880315020767),
 ('feature_n_staff', 0.018517794517554757),
 ('feature_p_v', 0.013803749904757445),
 ('feature_n_v', 0.013282975017032595),
 ('feature_p_noth', 0.012155106219173872),
 ('feature_n_hotel', 0.010235203493887195),
 ('feature_n_b', 0.009572667786459448),
 ('feature_n_u', 0.008780508703744864),
 ('feature_p_u', 0.008252742716570899),
 ('feature_p_room', 0.007129985895633),
 ('feature_n_clean', 0.006003562940194162),
 ('feature_p_com', 0.005957907587482291),
 ('feature_p_st', 0.00548804291493209),
 ('feature_n_bed', 0.004986141146327934),
 ('feature_n_g', 0.00490456934226588),
 ('feature_n_poor', 0.004885960006262271),
 ('feature_p_c', 0.004857032282268054),
 ('feature_n_small', 0.0046086657232627835),
 ('feature_n_n', 0.0038363601407059205),
 ('feature_n_hot', 0.0038314949397425654),
 ('feat

In [16]:
# Extract only the top 1k important features
adjusted_X_train = X_train.drop([f[0] for f in imp_fea[1000:]], 1)
adjusted_X_test = X_test.drop([f[0] for f in imp_fea[1000:]], 1)

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [17]:
%%time
# XGBoost Model training (full features)
xgb_model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05, max_depth=3)
xgb_model.fit(X_train, y_train)
print("model msr:", mean_squared_error(xgb_model.predict(X_test), y_test.astype('float')))

model msr: 1.472844140007202
Wall time: 9min 6s


In [18]:
%%time
# XGBoost Model training (top 1000 features)
xgb_model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05, max_depth=3)
xgb_model.fit(adjusted_X_train, y_train)
print("model msr:", mean_squared_error(xgb_model.predict(adjusted_X_test), y_test.astype('float')))

model msr: 1.4517227853842232
Wall time: 3min


In [20]:
# XGBoost Model hypro parameter tuning
adjusted_X_train_2, adjusted_X_validation, y_train_2, y_validation = train_test_split(adjusted_X_train,y_train,test_size=0.2, random_state=311)
for n_estimator in [100, 500, 1000, 2000, 5000]:
    for learning_rate in [0.01, 0.05, 0.1, 0.5, 1]:
        for max_depth in [3, 5, 7, 10, 20]:
            xgb_model = GradientBoostingRegressor(n_estimators=n_estimator, learning_rate=learning_rate, max_depth=max_depth)
            xgb_model.fit(adjusted_X_train_2, y_train_2)
            print(n_estimator, learning_rate, max_depth, "model msr:", mean_squared_error(xgb_model.predict(adjusted_X_validation), y_validation.astype('float')))

100 0.01 3 model msr: 2.179150058587754
100 0.01 5 model msr: 2.018214234285366
100 0.01 7 model msr: 1.9598600540397038
100 0.01 10 model msr: 1.9151585487430085
100 0.01 20 model msr: 1.9666079881180658
100 0.05 3 model msr: 1.6091307450175023
100 0.05 5 model msr: 1.5093224161522762
100 0.05 7 model msr: 1.4986661243248973
100 0.05 10 model msr: 1.4822656331882762
100 0.05 20 model msr: 1.7873286298005056
100 0.1 3 model msr: 1.5116883529526648
100 0.1 5 model msr: 1.4607314144019639
100 0.1 7 model msr: 1.4512961073976962
100 0.1 10 model msr: 1.4930722128266831
100 0.1 20 model msr: 1.7850429948338824
100 0.5 3 model msr: 1.5802710314670554
100 0.5 5 model msr: 1.7066935685302393
100 0.5 7 model msr: 1.8503673499267306
100 0.5 10 model msr: 1.9480183970941896
100 0.5 20 model msr: 2.0504884623118698
100 1 3 model msr: 2.0722017810926316
100 1 5 model msr: 2.5727250660939935
100 1 7 model msr: 2.9587989591689956
100 1 10 model msr: 3.073342815142272
100 1 20 model msr: 3.0326397928

In [21]:
%%time
# XGBoost Model model evaluation
xgb_model = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01, max_depth=3)
xgb_model.fit(adjusted_X_train, y_train)
print(n_estimator, learning_rate, max_depth, "model msr:", mean_squared_error(xgb_model.predict(adjusted_X_test), y_test.astype('float')))

5000 1 20 model msr: 1.4321332220719383
Wall time: 6min 29s


In [27]:
import pickle
import joblib

# Dump model and features
model_info = {"model": xgb_model, "features": imp_fea[:1000]}
joblib.dump(model_info, 'cs410_final_mmodel_info.joblib') 

['cs410_final_mmodel_info.joblib']

In [44]:
import pickle
import joblib
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))

loaded_model_info = joblib.load('cs410_final_mmodel_info.joblib')
in_str = ["good room", "bad weather"]
single_X = pd.DataFrame([in_str], columns=['Negative_Review', 'Positive_Review'])

# Remove stopwords
single_X['Positive_Review'] = single_X['Positive_Review'].apply(
    lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))
single_X['Negative_Review'] = single_X['Negative_Review'].apply(
    lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))

# Stemming
ps = PorterStemmer()
single_X['Positive_Review'] = single_X['Positive_Review'].apply(
    lambda x: ' '.join([ps.stem(y) for y in x.split()]))
single_X['Negative_Review'] = single_X['Negative_Review'].apply(
    lambda x: ' '.join([ps.stem(y) for y in x.split()]))

for ft in loaded_model_info["features"]:
    is_p = ft[0][8] == "p"
    f = ' '.join(ft[0][10:].split("__"))
    if is_p:
        single_X["feature_p_" + "__".join(f.split())] = single_X['Positive_Review'].apply(lambda x: f in x)
    else:
        single_X["feature_n_" + "__".join(f.split())] = single_X['Negative_Review'].apply(lambda x: f in x)

single_X = single_X.drop(["Positive_Review", "Negative_Review"], 1)

loaded_model_info["model"].predict(single_X)

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
D:\programs\Anaconda3\envs\cs410\lib\site-packages\sklearn\base.py:488: FutureWarn

array([7.7319033])

1000
66
